In [2]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as Dasasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class vgg_layers(nn.Module):
    def __init__(self, config: list, normalize: bool = True, in_channels: int = 3):
        super(vgg_layers, self).__init__()

        layers = []

        for out_channels in config:
            assert out_channels == "M" or isinstance(out_channels, int)

            if out_channels == "M":
                layers.append(
                    nn.MaxPool2d(
                        kernel_size=(2, 2),
                        stride=(1, 1),
                        padding=(1, 1)
                    )
                )
            else:
                layers.append(
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=3,
                        stride=1,
                        padding=1
                    )
                )
                if normalize:
                    layers.append(
                        nn.BatchNorm2d(
                            num_features=out_channels
                        )
                    )
                layers.append(
                    nn.ReLU(
                        inplace=True
                    )
                )
                in_channels = out_channels

        self.model = nn.Sequential(*layers)

    def forward(self, x):

        return self.model(x)

In [ ]:
class VGG(nn.Module):
    def __init__(self, features, out_features: int = 1000):
        super(VGG, self).__init__()

        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d(
            output_size=(7, 7)
        )
        self.classifier = nn.Sequential(
            nn.Linear(
                in_features=512*7*7,
                out_features=4096
            ),
            nn.ReLU(inplace=True),
            nn.Dropout(
                p=0.5
            ),
            
            nn.Linear(
                in_features=4096,
                out_features=4096
            ),
            nn.ReLU(inplace=True),
            nn.Dropout(
                p=0.5
            ),
            nn.Linear(
                in_features=4096,
                out_features=out_features
            )
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)

        h = x.view(-1, 512*7*7)

        x = self.classifier(h)
        return x, h

In [4]:
vgg11_config = [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"] # 8 + 3 = 11

vgg13_config = [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"] # 10 + 3 = 13

vgg16_config = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"] # 13 + 3 = 16

vgg19_config = [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] # 16 + 3 = 19